In [2]:
library(Biobase)
library(pcaMethods)
library(colorspace)
library(RColorBrewer)
library(sva)
library(DASC)
library(MASS)
library(NMF)
library(dendextend)
library(amap)
library(pvclust)
library(Rtsne)
library(vioplot)

In [10]:
date_sra = '2018_5_1'
dir_ensembl = '/Users/kf/Dropbox/kfdata/02_Data/my_db/Ensembl/release-91/'
dir_curated_transcriptome = paste0(dir_ensembl, 'curated_transcriptome/', date_sra, '/')
dir_tc = paste0(dir_curated_transcriptome, 'tc/')
dir_sra = paste0(dir_curated_transcriptome, 'sra/')
dir_sva = paste0(dir_curated_transcriptome, 'sva/')
#species = 'Anolis_carolinensis'
#infile = paste0(dir_ensembl, 'kallisto_summary/', date_sra, '/tpm.masked.kallisto.gene.log.tsv/', species, '.gene.log.tsv')
#srafile = paste0(dir_ensembl, 'sra/', date_sra, '/sra_table_mapped_', date_sra, '.tsv')
outdir = '/Users/kf/Dropbox/kfdata/02_Data/04_Convergence_Duplication/20180207_kallisto/'
dist_method="pearson"
min_dif = 0
selected_tissues = strsplit('brain|heart|kidney|liver|ovary|testis', '\\|')[[1]]

if (!endsWith(outdir, '/')) {
    outdir = paste0(outdir, '/')
}

# set directory
if (!file.exists(outdir)) {
    dir.create(outdir)
}
setwd(outdir)

In [41]:
tc_sra_intersect = function(tc, sra) {
    sra_run = sra$run
    tc = tc[,colnames(tc) %in% sra_run]
    sra = sra[sra$run %in% colnames(tc), ]
    return(list(tc=tc, sra=sra))
}

remove_nonexpressed_gene = function(tc) {
    gene_sum = apply(tc, 1, sum)
    tc_ex = tc[gene_sum > 0,]
    tc_ne = tc[gene_sum == 0,]
    return(list(tc_ex=tc_ex, tc_ne=tc_ne))
}

add_color_to_sra = function(sra, selected_tissues) {
    sra = sra[,(!names(sra) %in% c('bp_color','sp_color','tissue_color'))]
    bioproject = as.character(sra$bioproject)
    scientific_name = as.character(sra$scientific_name)
    tissue = as.character(sra$tissue)
    if (length(selected_tissues) <= 8) {
        tissue_color = brewer.pal(length(unique(tissue)), "Dark2")
        bp_color = rainbow_hcl(length(unique(bioproject)), c=50)
        sp_color = rainbow_hcl(length(unique(scientific_name)), c=100)
    } else if (length(selected_tissues) <= 12) {
        tissue_color = brewer.pal(length(unique(tissue)), "Paired")
        bp_color = rainbow_hcl(length(unique(bioproject)), c=50)
        sp_color = rainbow_hcl(length(unique(scientific_name)), c=100)
    } else {
        tissue_color = rainbow_hcl(length(selected_tissues), c=100)
        bp_color = rainbow_hcl(length(unique(bioproject)), c=50)
        sp_color = rainbow_hcl(length(unique(scientific_name)), c=150)
    }
    df_tissue = data.frame(tissue=sort(unique(tissue)), tissue_color=tissue_color[1:length(sort(unique(tissue)))], stringsAsFactors=FALSE)
    df_bp = data.frame(bioproject=sort(unique(bioproject)), bp_color=bp_color[1:length(sort(unique(bioproject)))], stringsAsFactors=FALSE)
    df_sp = data.frame(scientific_name=sort(unique(scientific_name)), sp_color=sp_color[1:length(sort(unique(scientific_name)))], stringsAsFactors=FALSE)
    sra = merge(sra, df_bp, sort=FALSE, all.y=FALSE)
    sra = merge(sra, df_sp, sort=FALSE, all.y=FALSE)
    sra = merge(sra, df_tissue, sort=FALSE, all.y=FALSE)
    return(sra)
}

sort_tc_and_sra = function(tc, sra, sort_columns=c("tissue","scientific_name","bioproject")) {
  for (column in rev(sort_columns)) {
    sra = sra[order(sra[column]),]
  }
  tc = tc[,sra$run[sra$run %in% colnames(tc)]]
  return(list(tc=tc, sra=sra))
}

sort_averaged_tc = function(tc) {
    split_colnames = strsplit(colnames(tc), "_")
    genus_names = c()
    specific_names = c()
    tissue_names = c()
    for (i in 1:length(split_colnames)) {
        genus_names = c(genus_names, split_colnames[[i]][1])
        specific_names = c(specific_names, split_colnames[[i]][2])
        tissue_names = c(tissue_names, split_colnames[[i]][3])
    }
    colname_order = order(tissue_names, genus_names, specific_names)
    tc = tc[, colname_order]
    return(tc)
}

cleanY = function(y, mod, svs) {
  X = cbind(mod, svs)
  Hat = solve(t(X) %*% X) %*% t(X)
  beta = (Hat %*% t(y))
  P = ncol(mod)
  return(y - t(as.matrix(X[,-c(1:P)]) %*% beta[-c(1:P),]))
}

map_color = function(redundant_variables, c) {
    uniq_var = unique(redundant_variables)
    uniq_col = rainbow_hcl(length(uniq_var), c=c)
    df_unique = data.frame(var=uniq_var, col=uniq_col, stringsAsFactors=FALSE)
    df_redundant = data.frame(var=redundant_variables, order=seq(1, length(redundant_variables)), stringsAsFactors=FALSE)
    df_redundant = merge(df_redundant, df_unique, by="var", all.x=TRUE, stringsAsFactors=FALSE)
    df_redundant = df_redundant[order(df_redundant$order),]
    return(df_redundant$col)
}

color_children2parent = function(node) {
    if (length(node)==2) {
        child1_color = attributes(node[[1]])$edgePar[['col']]
        child2_color = attributes(node[[2]])$edgePar[['col']]
        if ((!is.null(child1_color))&(!is.null(child2_color))) {
            if (child1_color==child2_color) {
                attributes(node)$edgePar[['col']] = child1_color
            }
        }
    }
    return(node)
}

draw_dendrogram = function(sra, tc_dist_dist, fontsize=7) {
    dend <- as.dendrogram(hclust(tc_dist_dist))
    dend_colors = sra$tissue_color[order.dendrogram(dend)]
    labels_colors(dend) <- dend_colors
    dend_labels <- sra$run[order.dendrogram(dend)]
    dend <- color_branches(dend, labels=dend_labels, col=dend_colors)
    dend <- set(dend, "branches_lwd", 1)
    for (i in 1:ncol(tc)) {
        dend = dendrapply(dend, color_children2parent)
    }
    cex.xlab = min(fontsize, max(0.2, 0.5/log10(nrow(sra))))
    par(cex=cex.xlab)
    plot(dend, las=1, axes=FALSE)
    par(cex=1)
    axis(side=2, line=0, las=1)
    mtext('Distance', side=2, line=8.5, outer=FALSE)
    n = nrow(sra)
    symbols(1:n, rep(0,n), circles=rep(1,n), add=TRUE, inches=0.02, xpd=TRUE, lwd=1, 
            bg=sra$tissue_color[order.dendrogram(dend)], fg=sra$bp_color[order.dendrogram(dend)])
}

draw_pca = function(sra, tc_dist_matrix, fontsize=7) {
    set.seed(1)
    pca = prcomp(tc_dist_matrix)
    xlabel = paste0("PC 1 (", round(summary(pca)$importance[2,1]*100, digits=1), "%)")
    ylabel = paste0("PC 2 (", round(summary(pca)$importance[2,2]*100, digits=1), "%)")
    plot(pca$rotation[,1], pca$rotation[,2], pch=21, cex=2, lwd=1, bg=sra$tissue_color, col=sra$bp_color, xlab=xlabel, ylab=ylabel, las=1)
    #plot(pca$x[,1], pca$x[,2], pch=21, cex=2, lwd=2, bg=sra$tissue_color, col=sra$bp_color, main=title, xlab=xlabel, ylab=ylabel, las=1)
}

draw_mds = function(sra, tc_dist_dist, fontsize=7) {
    set.seed(1)
    try_out = tryCatch(
        {isoMDS(tc_dist_dist, k=2, maxit=100)},
        error = function(a){return("MDS failed.")}
    )
    if (mode(try_out)=="character") {
        cat('MDS failed.\n')
        plot(c(0, 1), c(0, 1), ann = F, bty = 'n', type = 'n', xaxt = 'n', yaxt = 'n')
    } else {
        mds <- try_out
        plot(mds$points[,1], mds$points[,2], pch=21, cex=2, lwd=1, bg=sra$tissue_color, col=sra$bp_color, xlab="MDS dimension 1", ylab="MDS dimension 2", las=1)
    }
}

draw_tau_histogram = function(tc, sra, selected_tissues, fontsize=7) {
    df_tau = tissue2tau(tissue_mean(tc, sra, selected_tissues), rich.annotation=FALSE, unlog=TRUE)
    hist_out = hist(df_tau$tau, breaks=seq(0,1,0.05), las=1, xlab='Tau (expression specificity)', ylab='Gene count', main='', col='gray')
    num_noexp = sum(is.na(df_tau$tau))
    num_all = nrow(df_tau)
    #num_exp = nrow(df_tau) - num_noexp
    #text_noexp = paste('Expressed genes:', num_exp, '\nNon-expressed genes:', num_noexp)
    text_noexp = paste0('Excluded due to\nno expression:\n', num_noexp, '/', num_all, ' genes')
    text(0, max(hist_out$counts)*0.85, text_noexp, pos=4)
}

draw_exp_level_histogram = function(tc, sra, selected_tissues, fontsize=7) {
    tc_tissue = tissue_mean(tc, sra, selected_tissues)
    xmax = apply(tc_tissue, 1, max)
    xmax[xmax<0] = 0
    xmax[xmax>15] = 15
    breaks = seq(0, 15, 1)
    hist_out = hist(xmax, breaks=breaks, las=1, xlab='Max expression (log TPM+1)', ylab='Gene count', main='', col='gray')
}

draw_legend = function(sra, new=TRUE, pos="center", fontsize=7, nlabel.in.col) {
    if (new) {
        plot.new()
    }
    tissue_unique = unique(sra$tissue)
    bp_unique = unique(sub(';.*', '', sra$bioproject))
    tissue_color_unique = unique(sra$tissue_color)
    bp_color_unique = unique(sra$bp_color)
    ncol = ceiling((length(tissue_unique)+length(bp_unique)+2)/nlabel.in.col)
    legend_text = c('Organ', as.character(tissue_unique), '', 'BioProject',as.character(bp_unique))
    legend_color = c(rgb(1,1,1,0), rep(rgb(1,1,1,0), length(tissue_color_unique)), rgb(1,1,1,0), rgb(1,1,1,0), bp_color_unique)
    legend_bg = c(rgb(1,1,1,0), tissue_color_unique, rgb(1,1,1,0), rgb(1,1,1,0), rep(rgb(1,1,1,0), length(bp_color_unique)))
    legend_font = c(2, rep(1, length(tissue_color_unique)), 1, 2, rep(1, length(bp_color_unique)))
    legend(pos, legend=legend_text, pch=21, lwd=1, lty=0, col=legend_color, pt.bg=legend_bg, text.font=legend_font, ncol=ncol, bty='n')
}

save_plot = function(tc, sra, sva_out, dist_method, file, selected_tissues, fontsize=7) {
    out = tc_sra_intersect(tc, sra) ; tc = out[['tc']] ; sra = out[['sra']]
    sra = add_color_to_sra(sra, selected_tissues)
    out = sort_tc_and_sra(tc, sra) ; tc = out[["tc"]] ; sra = out[["sra"]]
    pdf(paste0(file, ".pdf"), height=8, width=7.2, fonts='Helvetica', pointsize=fontsize)
    layout_matrix=matrix(c(
        2,2,2,2,2,1,1,1,1,1,1,1,
        2,2,2,2,2,1,1,1,1,1,1,1,
        2,2,2,2,2,1,1,1,1,1,1,1,
        2,2,2,2,2,1,1,1,1,1,1,1,
        2,2,2,2,2,1,1,1,1,1,1,1,
        3,3,3,4,4,4,5,5,5,6,6,6,
        3,3,3,4,4,4,5,5,5,6,6,6,
        3,3,3,4,4,4,5,5,5,6,6,6,
        7,7,7,8,8,8,9,9,9,9,9,9,
        7,7,7,8,8,8,9,9,9,9,9,9,
        7,7,7,8,8,8,9,9,9,9,9,9,
        7,7,7,8,8,8,9,9,9,9,9,9,
        10,10,10,10,10,10,10,10,10,10,10,10,
        10,10,10,10,10,10,10,10,10,10,10,10
        ),
      14, 12, byrow=TRUE
    )
    layout(layout_matrix)
    tc_dist_matrix = cor(tc, method=dist_method)
    tc_dist_matrix[is.na(tc_dist_matrix)] = 0
    tc_dist_dist = Dist(t(tc), method=dist_method) + 0.000000001
    tc_dist_dist[is.na(tc_dist_dist)] = 1
    par(mar=c(6,6,1,0)); draw_dendrogram(sra, tc_dist_dist, fontsize)
    par(mar=c(0,0,0,0)); draw_heatmap(sra, tc_dist_matrix, legend=FALSE)
    #draw_dendrogram(sra, tc, nboot=1000, cex.xlab=0.6, pvclust_file=paste0(file, '.pvclust.RData'))
    par(mar=c(4,4,0.1,1)); draw_pca(sra, tc_dist_matrix, fontsize)
    par(mar=c(4,4,0.1,1)); draw_mds(sra, tc_dist_dist, fontsize)
    par(mar=c(4,4,0.1,1)); draw_tsne(sra, tc, fontsize)
    par(mar=c(4,5,0.1,1)); draw_boxplot(sra, tc_dist_matrix, fontsize)
    par(mar=c(4,4,1,1)); draw_exp_level_histogram(tc, sra, selected_tissues, fontsize)
    par(mar=c(4,4,1,1)); draw_tau_histogram(tc, sra, selected_tissues, fontsize)
    par(mar=rep(0.1,4)); df_r2 = draw_sva_summary(sva_out, tc, sra, fontsize)
    if (! all(is.na(df_r2))) {
        write.table(df_r2, paste0(file,'.r2.tsv'), sep='\t', row.names=FALSE)
    }
    par(mar=rep(0.1,4)); draw_legend(sra, new=TRUE, pos="center", fontsize=fontsize, nlabel.in.col=8)
    graphics.off()
}

In [94]:
# heatmap

draw_heatmap2 = function(sra, tc_dist_matrix, legend=TRUE, cbar=TRUE, main=NA, fontsize=7) {
    bp_fac = factor(sub(';.*', '', sra[,c("bioproject")]))
    tissue_fac = factor(sra[,c("tissue")])
    ann_label = data.frame(bioproject=bp_fac, tissue=tissue_fac)
    bp_col_uniq = unique(sra$bp_color[order(sra$bioproject)])
    tissue_col_uniq = unique(sra$tissue_color[order(sra$tissue)])
    ann_color = list(bioproject=bp_col_uniq, tissue=tissue_col_uniq)
    breaks = c(0, seq(0.3, 1, 0.01))
    aheatmap(tc_dist_matrix, color="-RdYlBu2:71", main=main, labRow=NA, labCol=NA,
             Rowv=NA, Colv=NA, revC=TRUE, legend=cbar, breaks=breaks, 
             annCol=ann_label, annRow=ann_label, annColors=ann_color, annLegend=legend, fontsize=fontsize)
}

fontsize = 6
spp = c('Bos taurus','Homo sapiens','Gallus gallus','Macaca mulatta','Canis lupus','Sus scrofa','Ovis aries','Mus musculus','Callithrix jacchus',
'Rattus norvegicus','Monodelphis domestica','Oreochromis niloticus','Oryctolagus cuniculus','Danio rerio','Anolis carolinensis','Chinchilla lanigera',
'Astyanax mexicanus','Xenopus tropicalis','Oryzias latipes','Gadus morhua','Ornithorhynchus anatinus')
spp = spp[order(spp)]

pdf('paneled_species_heatmap.pdf', height=6.5, width=7.2, fonts='Helvetica', pointsize=fontsize)
par(mfrow=c(4,6), mar=c(0,0,0,0))

for (sp in spp) {
    cat(sp, '\n')
    sp_filled = sub(' ', '_', sp)
    
    files = list.files(dir_sra)
    file = files[grep(sp_filled, files)]
    sra = read.table(paste0(dir_sra, file), sep='\t', header=TRUE, comment.char='', check.names=FALSE, quote='', stringsAsFactors=FALSE)
    
    files = list.files(dir_tc)
    file = files[grep(sp_filled, files)]
    tc = read.table(paste0(dir_tc, file), sep='\t', header=TRUE, quote='', stringsAsFactors=FALSE)

    sra = add_color_to_sra(sra, selected_tissues)
    out = tc_sra_intersect(tc, sra) ; tc = out[['tc']] ; sra = out[['sra']]
    out = sort_tc_and_sra(tc, sra) ; tc = out[["tc"]] ; sra = out[["sra"]]

    tc_dist_matrix = cor(tc, method=dist_method)
    tc_dist_matrix[is.na(tc_dist_matrix)] = 0
    
    main = substitute(italic(x), list(x=sp))
    
    draw_heatmap2(sra, tc_dist_matrix, legend=FALSE, cbar=FALSE, main=main, fontsize=fontsize)
    
    #if (sp=='Bos taurus') {
    #    break
    #}
}
graphics.off()
cat('done.\n')

Anolis carolinensis 
Astyanax mexicanus 
Bos taurus 
Callithrix jacchus 
Canis lupus 
Chinchilla lanigera 
Danio rerio 
Gadus morhua 
Gallus gallus 
Homo sapiens 
Macaca mulatta 
Monodelphis domestica 
Mus musculus 
Oreochromis niloticus 
Ornithorhynchus anatinus 
Oryctolagus cuniculus 
Oryzias latipes 
Ovis aries 
Rattus norvegicus 
Sus scrofa 
Xenopus tropicalis 
done.


In [ ]:
# t-SNE

draw_tsne2 = function(sra, tc, fontsize=7, main=NA) {
    perplexity = min(30, floor(nrow(sra)/4))
    set.seed(1)
    out_tsne = Rtsne(as.matrix(t(tc)), theta=0, check_duplicates=FALSE, verbose=FALSE, perplexity=perplexity, dims=2)
    try_out = tryCatch(
        {
            plot(out_tsne$Y[,1], out_tsne$Y[,2], pch=21, cex=2, lwd=1, bg=sra$tissue_color, col=sra$bp_color, 
                 xaxt='n', yaxt='n', las=1, main=main)
        },
        error = function(a){return("t-SNE plot failed.")}
    )
    if (mode(try_out)=="character") {
        cat('t-SNE failed.\n')
        plot(c(0, 1), c(0, 1), ann = F, bty = 'n', type = 'n', xaxt = 'n', yaxt = 'n')
    }    
}

fontsize = 8
spp = c('Bos taurus','Homo sapiens','Gallus gallus','Macaca mulatta','Canis lupus','Sus scrofa','Ovis aries','Mus musculus','Callithrix jacchus',
'Rattus norvegicus','Monodelphis domestica','Oreochromis niloticus','Oryctolagus cuniculus','Danio rerio','Anolis carolinensis','Chinchilla lanigera',
'Astyanax mexicanus','Xenopus tropicalis','Oryzias latipes','Gadus morhua','Ornithorhynchus anatinus')
spp = spp[order(spp)]

pdf('paneled_species_t-SNE.pdf', height=5.5, width=7.2, fonts='Helvetica', pointsize=fontsize)
par(mfrow=c(4,6), mar=c(0,0,0,0))

for (sp in spp) {
    cat(sp, '\n')
    sp_filled = sub(' ', '_', sp)
    
    files = list.files(dir_sra)
    file = files[grep(sp_filled, files)]
    sra = read.table(paste0(dir_sra, file), sep='\t', header=TRUE, comment.char='', check.names=FALSE, quote='', stringsAsFactors=FALSE)
    
    files = list.files(dir_tc)
    file = files[grep(sp_filled, files)]
    tc = read.table(paste0(dir_tc, file), sep='\t', header=TRUE, quote='', stringsAsFactors=FALSE)

    sra = add_color_to_sra(sra, selected_tissues)
    out = tc_sra_intersect(tc, sra) ; tc = out[['tc']] ; sra = out[['sra']]
    out = sort_tc_and_sra(tc, sra) ; tc = out[["tc"]] ; sra = out[["sra"]]
    
    main = substitute(italic(x), list(x=sp))
    
    par(mar=c(0.2,0.2,2,0.2)); draw_tsne2(sra=sra, tc=tc, main=main, fontsize=fontsize)
    
    #if (sp=='Bos taurus') {
    #    break
    #}
}
graphics.off()
cat('done.\n')

Anolis carolinensis 
Astyanax mexicanus 
Bos taurus 
Callithrix jacchus 
Canis lupus 
Chinchilla lanigera 
Danio rerio 


In [158]:
# correlation boxplot

draw_boxplot2 = function(sra, tc_dist_matrix, main, fontsize=7) {
    is_same_bp = outer(sra$bioproject, sra$bioproject, function(x,y){x==y})
    is_same_tissue = outer(sra$tissue, sra$tissue, function(x,y){x==y})
    is_same_entry = outer(1:nrow(sra), 1:nrow(sra), function(x,y){x==y})
    plot(c(0.5, 4.5), c(0, 1), main=main, type = 'n', xlab='', ylab="Pearson's correlation\ncoefficient", las=1, xaxt='n')
    boxplot(tc_dist_matrix[(!is_same_bp)&(!is_same_tissue)&(!is_same_entry)], at=1, add=TRUE, col='gray', yaxt='n')
    boxplot(tc_dist_matrix[(is_same_bp)&(!is_same_tissue)&(!is_same_entry)], at=2, add=TRUE, col='gray', yaxt='n')
    boxplot(tc_dist_matrix[(!is_same_bp)&(is_same_tissue)&(!is_same_entry)], at=3, add=TRUE, col='gray', yaxt='n')
    boxplot(tc_dist_matrix[(is_same_bp)&(is_same_tissue)&(!is_same_entry)], at=4, add=TRUE, col='gray', yaxt='n')
    labels = c('bw\nbw', 'bw\nwi', 'wi\nbw', 'wi\nwi')
    axis(side=1, at=c(1,2,3,4), labels=labels, padj=0.5)
    axis(side=1, at=0.35, labels='Organ\nBioProject', padj=0.5, hadj=1, tick=FALSE)
}

fontsize = 8
spp = c('Bos taurus','Homo sapiens','Gallus gallus','Macaca mulatta','Canis lupus','Sus scrofa','Ovis aries','Mus musculus','Callithrix jacchus',
'Rattus norvegicus','Monodelphis domestica','Oreochromis niloticus','Oryctolagus cuniculus','Danio rerio','Anolis carolinensis','Chinchilla lanigera',
'Astyanax mexicanus','Xenopus tropicalis','Oryzias latipes','Gadus morhua','Ornithorhynchus anatinus')
spp = spp[order(spp)]

pdf('paneled_species_boxplot.pdf', height=5.5, width=7.2, fonts='Helvetica', pointsize=fontsize)
par(mfrow=c(4,6), mar=c(0,0,0,0))

for (sp in spp) {
    cat(sp, '\n')
    sp_filled = sub(' ', '_', sp)
    
    files = list.files(dir_sra)
    file = files[grep(sp_filled, files)]
    sra = read.table(paste0(dir_sra, file), sep='\t', header=TRUE, comment.char='', check.names=FALSE, quote='', stringsAsFactors=FALSE)
    
    files = list.files(dir_tc)
    file = files[grep(sp_filled, files)]
    tc = read.table(paste0(dir_tc, file), sep='\t', header=TRUE, quote='', stringsAsFactors=FALSE)

    sra = add_color_to_sra(sra, selected_tissues)
    out = tc_sra_intersect(tc, sra) ; tc = out[['tc']] ; sra = out[['sra']]
    out = sort_tc_and_sra(tc, sra) ; tc = out[["tc"]] ; sra = out[["sra"]]

    tc_dist_matrix = cor(tc, method=dist_method)
    tc_dist_matrix[is.na(tc_dist_matrix)] = 0
    
    main = substitute(italic(x), list(x=sp))
    
    par(mar=c(4,5,2,1)); draw_boxplot2(sra, tc_dist_matrix, main, fontsize)    
    
    #if (sp=='Anolis carolinensis') {
    #    break
    #}
}
graphics.off()
cat('done.\n')

Anolis carolinensis 
Astyanax mexicanus 
Bos taurus 
Callithrix jacchus 
Canis lupus 
Chinchilla lanigera 


Warning message in min(x):
“no non-missing arguments to min; returning Inf”Warning message in max(x):
“no non-missing arguments to max; returning -Inf”Warning message in min(x):
“no non-missing arguments to min; returning Inf”Warning message in max(x):
“no non-missing arguments to max; returning -Inf”Warning message in min(x):
“no non-missing arguments to min; returning Inf”Warning message in max(x):
“no non-missing arguments to max; returning -Inf”

Danio rerio 
Gadus morhua 
Gallus gallus 
Homo sapiens 
Macaca mulatta 
Monodelphis domestica 
Mus musculus 
Oreochromis niloticus 
Ornithorhynchus anatinus 
Oryctolagus cuniculus 
Oryzias latipes 
Ovis aries 
Rattus norvegicus 
Sus scrofa 
Xenopus tropicalis 
done.


In [125]:
# predictor of surrogate variables

draw_sva_summary2 = function(sva_out, tc, sra, main, fontsize) {
    if ((is.null(sva_out))|(class(sva_out)=='try-error')) {
        plot(c(0, 1), c(0, 1), ann = F, bty = 'n', type = 'n', xaxt = 'n', yaxt = 'n', main=main)
        df = NA
    } else {
        out = tc_sra_intersect(tc, sra) ; tc = out[['tc']] ; sra = out[['sra']]
        out = sort_tc_and_sra(tc, sra) ; tc = out[["tc"]] ; sra = out[["sra"]]
        sra$fraction_lost_fastp = 1 - (sra$num_read_fastp / sra$num_read_unfiltered)
        sra$fraction_lost_mask = 1 - (sra$num_read_mask / sra$num_read_fastp)
        cols = c('tissue','bioproject','lib_selection','layout','instrument','num_read_masked','fraction_lost_fastp',
                 'fraction_lost_mask','min_read_len_masked','avg_read_len_masked','max_read_len_masked','mapping_rate_masked')
        label_cols = c('organ','BioProject','library selection','library layout','instrument','number of read','% lost, fastp',
                       '% lost, misc feature','minimum read length','average read length','maximum read length','mapping rate')
        num_sv = sva_out$n.sv
        df = data.frame(matrix(NA, num_sv, length(cols)))
        colnames(df) = cols
        rownames(df) = paste0('SV', 1:nrow(df))
        for (i in 1:length(cols)) {
            for (j in 1:num_sv) {
                if (length(unique(sra[,cols[i]]))==1) {
                    df[j,i] = NA
                } else {
                    df[j,i] = summary(lm(sva_out$sv[,j] ~ sra[,cols[i]]))$adj.r.squared
                }
            }
        }
        colnames(df) = label_cols
        breaks = seq(0, 1, 0.02)
        colors = colorRampPalette(c("blue", "yellow", "red"))(length(breaks))
        df2 = t(df)
        df2[df2<0] = 0
        aheatmap(df2, color=colors, Rowv=NA, Colv=NA, revC=TRUE, main=main, legend=FALSE,
                 breaks=breaks, fontsize=fontsize, labRow=NA)
    }
    return(df)
}

fontsize = 6
spp = c('Bos taurus','Homo sapiens','Gallus gallus','Macaca mulatta','Canis lupus','Sus scrofa','Ovis aries','Mus musculus','Callithrix jacchus',
'Rattus norvegicus','Monodelphis domestica','Oreochromis niloticus','Oryctolagus cuniculus','Danio rerio','Anolis carolinensis','Chinchilla lanigera',
'Astyanax mexicanus','Xenopus tropicalis','Oryzias latipes','Gadus morhua','Ornithorhynchus anatinus')
spp = spp[order(spp)]

pdf('paneled_species_SV_predictor.pdf', height=7, width=7.2, fonts='Helvetica', pointsize=fontsize)
#layout_matrix=matrix(c(
#    1,2,3,4,5,6,0,
#    7,8,9,10,11,12,0,
#    13,14,15,16,17,18,0,
#    19,20,21,22,23,24,0
#    ),
#  4, 7, byrow=TRUE
#)
#layout(layout_matrix)
par(mfrow=c(4,6), mar=c(0,0,0,0))

for (sp in spp) {
    cat(sp, '\n')
    sp_filled = sub(' ', '_', sp)
    
    files = list.files(dir_sva)
    files = files[grep(sp_filled, files)]
    file = files[length(files)]
    load(paste0(dir_sva, file))
    
    files = list.files(dir_sra)
    file = files[grep(sp_filled, files)]
    sra = read.table(paste0(dir_sra, file), sep='\t', header=TRUE, comment.char='', check.names=FALSE, quote='', stringsAsFactors=FALSE)    
    
    files = list.files(dir_tc)
    file = files[grep(sp_filled, files)]
    tc = read.table(paste0(dir_tc, file), sep='\t', header=TRUE, quote='', stringsAsFactors=FALSE)

    sra = add_color_to_sra(sra, selected_tissues)
    out = tc_sra_intersect(tc, sra) ; tc = out[['tc']] ; sra = out[['sra']]
    out = sort_tc_and_sra(tc, sra) ; tc = out[["tc"]] ; sra = out[["sra"]]
    
    main = substitute(italic(x), list(x=sp))
    
    par(mar=rep(0.1,4)); df_r2 = draw_sva_summary2(sva_out, tc, sra, main, fontsize)
    
    #if (sp=='Anolis carolinensis') {
    #    break
    #}
}
graphics.off()
cat('done.\n')

Anolis carolinensis 
Astyanax mexicanus 
Bos taurus 
Callithrix jacchus 
Canis lupus 
Chinchilla lanigera 
Danio rerio 
Gadus morhua 
Gallus gallus 
Homo sapiens 
Macaca mulatta 
Monodelphis domestica 
Mus musculus 
Oreochromis niloticus 
Ornithorhynchus anatinus 
Oryctolagus cuniculus 
Oryzias latipes 
Ovis aries 
Rattus norvegicus 
Sus scrofa 
Xenopus tropicalis 
done.


In [155]:
# number of RNA-seq, bioproject, and surrogate variable

fontsize = 8
spp = c('Bos taurus','Homo sapiens','Gallus gallus','Macaca mulatta','Canis lupus','Sus scrofa','Ovis aries','Mus musculus','Callithrix jacchus',
'Rattus norvegicus','Monodelphis domestica','Oreochromis niloticus','Oryctolagus cuniculus','Danio rerio','Anolis carolinensis','Chinchilla lanigera',
'Astyanax mexicanus','Xenopus tropicalis','Oryzias latipes','Gadus morhua','Ornithorhynchus anatinus')
spp = spp[order(spp)]

pdf('paneled_species_num_data.pdf', height=3, width=4.5, fonts='Helvetica', pointsize=fontsize)
layout_matrix=matrix(c(1,1,1,1,1,2,2,2,3,3,3), 1, 11, byrow=TRUE)
layout(layout_matrix)

num_exp = c()
num_bioproject = c()
num_sv = c()
for (sp in spp) {
    cat(sp, '\n')
    sp_filled = sub(' ', '_', sp)
    
    files = list.files(dir_sva)
    files = files[grep(sp_filled, files)]
    file = files[length(files)]
    load(paste0(dir_sva, file))
    
    files = list.files(dir_sra)
    file = files[grep(sp_filled, files)]
    sra = read.table(paste0(dir_sra, file), sep='\t', header=TRUE, comment.char='', check.names=FALSE, quote='', stringsAsFactors=FALSE)    
    sra = sra[(sra$exclusion=='no'),]
    
    num_exp = c(num_exp, nrow(sra))
    names(num_exp)[length(num_exp)] = sp
    
    num_bioproject = c(num_bioproject, length(unique(sra[['bioproject']])))
    names(num_bioproject)[length(num_bioproject)] = sp
    
    if ('n.sv' %in% names(sva_out)) {
        num_sv = c(num_sv, sva_out[['n.sv']])
    } else {
        num_sv = c(num_sv, 0)
    }
    names(num_sv)[length(num_sv)] = sp
}

ind = order(num_exp)
num_exp = num_exp[ind]
num_bioproject = num_bioproject[ind]
num_sv = num_sv[ind]

par(ps=fontsize, mar=c(4,10,0.1,1))
barplot(num_exp, las=1, col='black', xlab='Number of RNA-seq experiments', horiz=TRUE, xlim=c(0,400), font=3, xaxt='n')
axis(1)
par(ps=fontsize, mar=c(4,0.1,0.1,1))
barplot(num_bioproject, las=1, col='black', xlab='Number of BioProjects', yaxt='n', horiz=TRUE, xlim=c(0,40))
barplot(num_sv, las=1, col='black', xlab='Number of sorrogate variables', yaxt='n', horiz=TRUE, xlim=c(0,30))


graphics.off()
cat('done.\n')

Anolis carolinensis 
Astyanax mexicanus 
Bos taurus 
Callithrix jacchus 
Canis lupus 
Chinchilla lanigera 
Danio rerio 
Gadus morhua 
Gallus gallus 
Homo sapiens 
Macaca mulatta 
Monodelphis domestica 
Mus musculus 
Oreochromis niloticus 
Ornithorhynchus anatinus 
Oryctolagus cuniculus 
Oryzias latipes 
Ovis aries 
Rattus norvegicus 
Sus scrofa 
Xenopus tropicalis 
done.


In [124]:
graphics.off()
